In [1]:
!gdown '1fuPqJy8j8O_Xm2a46aenKlr31I_GGwXV'

Downloading...
From: https://drive.google.com/uc?id=1fuPqJy8j8O_Xm2a46aenKlr31I_GGwXV
To: /content/data_clean_v2.csv
100% 1.75M/1.75M [00:00<00:00, 31.4MB/s]


In [7]:
import pandas as pd

file_path = 'data_clean_v2.csv'
data = pd.read_csv(file_path)


missing_values = data.isna().sum()
missing_values[missing_values > 0]
data.drop(columns = ['index'], inplace = True)
data.columns


data.info()
df = data.drop(columns=['Salary','decription']).copy()
df.reset_index(inplace = True)


df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   location      2138 non-null   object 
 1   companyType   2138 non-null   object 
 2   decription    2138 non-null   object 
 3   salaryType    2138 non-null   object 
 4   contractType  2138 non-null   object 
 5   jobType       2138 non-null   object 
 6   experience    1505 non-null   object 
 7   gender        2138 non-null   object 
 8   vacacies      2138 non-null   int64  
 9   education     1510 non-null   object 
 10  minAge        2138 non-null   int64  
 11  maxAge        1936 non-null   float64
 12  skill         2138 non-null   object 
 13  Salary        2138 non-null   object 
 14  Partner       2138 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 250.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 14 

In [8]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
imputer = KNNImputer(n_neighbors = 2)

original_values = {}
columns_to_encode = df.select_dtypes(include = 'object').columns
columns = df.columns

def encode_row(row, encoders, original_values):
    for column in row.index:
        if pd.notna(row[column]):
            if column in encoders:
                label_encoder = encoders[column]
                row[column] = label_encoder.transform([row[column]])[0]
                if column not in original_values:
                    original_values[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    return row

In [11]:
df_encoded = df.copy()

encoders = {}
original_values = {}

for column in columns_to_encode:
    encoders[column] = LabelEncoder()
    encoders[column].fit(df[column].dropna().unique())

df_encoded = df_encoded.apply(lambda row: encode_row(row, encoders, original_values), axis=1)

df_encoded = imputer.fit_transform(df_encoded.drop([ 'index'], axis=1))

df_encoded = pd.DataFrame(df_encoded, columns = columns)

for column in columns_to_encode:
    df_encoded[column] = df_encoded[column].astype(int).map({v: k for k, v in original_values[column].items()})

df_encoded.isnull().sum()

ValueError: ignored

In [10]:
import math
df_encoded['maxAge'] = df_encoded['maxAge'].apply(math.ceil)


df_encoded['decription'] = data['decription'].copy()
df_encoded['Salary'] = data['Salary'].copy()

df_encoded.to_csv("data_clean_and_fill.csv", index = False)

ValueError: ignored

In [12]:
import joblib

joblib.dump(imputer, 'KNN_imputer_tong.pkl')

['KNN_imputer_tong.pkl']